In [1]:
import os

In [2]:
%pwd

'c:\\DataScience\\Projects\\Predictive_maintenance\\Predictive_Maintenance_JetEngine\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\DataScience\\Projects\\Predictive_maintenance\\Predictive_Maintenance_JetEngine'

In [7]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    X_train_path: Path
    y_train_path: Path
    X_test_path: Path
    ground_truth_data_apth: Path
    model_name: str
    criterion: str
    max_features: str
    n_estimators: int
    oob_score: bool

In [8]:
from src.PredictiveMaintenance.constants import *
from src.PredictiveMaintenance.utils.common import read_yaml, create_directories

In [9]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.RandomForestRegressor

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            X_train_path = config.X_train_path,
            y_train_path = config.y_train_path,
            X_test_path=config.X_test_path,
            ground_truth_data_apth=config.ground_truth_data_path,
            model_name = config.model_name,
            criterion=params.criterion,
            max_features=params.max_features,
            n_estimators=params.n_estimators,
            oob_score=params.oob_score
            
        )

        return model_trainer_config

In [10]:
import pandas as pd
import os
from src.PredictiveMaintenance import logger
from sklearn.ensemble import RandomForestRegressor
import joblib

In [11]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config


    
    def train(self):

        train_X = pd.read_csv(self.config.X_train_path)
        train_y = pd.read_csv(self.config.y_train_path)
        test_X = pd.read_csv(self.config.X_test_path)
        RUL=pd.read_csv(self.config.ground_truth_data_apth)
        

        lr = RandomForestRegressor(n_estimators=self.config.n_estimators, criterion=self.config.criterion, max_features=self.config.max_features, random_state=42)
        lr.fit(train_X, train_y)

        joblib.dump(lr, os.path.join(self.config.root_dir, self.config.model_name))

In [12]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2024-03-09 16:00:32,765: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-03-09 16:00:32,779: INFO: common: yaml file: params.yaml loaded successfully]
[2024-03-09 16:00:32,779: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-03-09 16:00:32,786: INFO: common: created directory at: artifacts]
[2024-03-09 16:00:32,786: INFO: common: created directory at: artifacts/model_trainer]


c:\Users\suzal\anaconda3\envs\envrul\lib\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
